# Методы расширения

Функциональность любого типа можно расширить при помощи механизма методов расширения

In [ ]:
// Так он должен, по-хорошему, объявляться.
public static class StringExtentions
{
    public static string AddBorders(this string str, char symbol='=', int size=3)
    {
        string border = new string(symbol, size);
        return border + str + border;
    }
}

**Ввиду особенностей Jupyter Notebook (он оборачивает всё в класс) я объявляю метод расширения прямо в ячейке. В реальности требуется публичный статический класс**

In [ ]:
//public static class StringExtentions
//{
    public static string AddBorders(this string str, char symbol='=', int size=3)
    {
        string border = new string(symbol, size);
        return border + str + border;
    }
//}

In [ ]:
string str = "Hello";
str.AddBorders()

# Перечисления (IEnumerable), перечислители (IEnumerator)

## 1. IEnumerable

Программы *очень* часто работают с **коллекциями** объектов. 

Наиболее простой вид коллекции это коллекция, элементы которой можно (как-то) **перечислить** (последовательность). Так как перечисление - наиболее базовая операция, алгоритмы, использующие только её можно применить на **любые** последовательности объектов. Это особенно эффектно в LINQ.

В шарпе все перечислимые коллекции реализуют интерфейс `IEnumerable<T>` или `IEnumerable`.
```csharp
public interface IEnumerable
{
    IEnumerator GetEnumerator();
}

public interface IEnumerable<out T> : IEnumerable
{
    IEnumerator<T> GetEnumerator();
}
```

Чтобы можно было итерироваться по собственным коллекциям (и таким образом дать возможность использовать всю мощь стандартной библиотеки), достаточно реализовать этот интерфейс.

Это довольно просто сделать, если "под капотом" у типа - другая коллекция / массив.

```csharp

```

In [ ]:
public class Box<T> : IEnumerable<T>
{
    private T[] items;

    public Box(params T[] items)
    {
        this.items = items;
    }

    public IEnumerator<T> GetEnumerator() => ((IEnumerable<T>)items).GetEnumerator();

    IEnumerator IEnumerable.GetEnumerator() => items.GetEnumerator();
    
    // Какая-то доп. логика коробки
}

## 2. foreach

Самый простой пример использования `IEnumerable` это `foreach`.

Оператор `foreach` позволяет итерироваться по любому объекту, **поддерживающему** метод
```csharp
IEnumerator GetEnumerator();
```
или
```csharp
IEnumerator<T> GetEnumerator();
```

In [ ]:
Box<int> box = new Box<int>(1, 2, 3, 4);

foreach(var item in box)
{
    Console.WriteLine(item);
}

На самом деле для `foreach`, главное - поддерживать нужный метод. 

Не обязательно реализовывать `IEnumerable`:

In [ ]:
public class NotEnumerableBox<T> // НЕТ ЯВНОГО УКАЗАНИЯ РЕАЛИЗАЦИИ
{
    private T[] items;

    public NotEnumerableBox(params T[] items)
    {
        this.items = items;
    }

    public IEnumerator<T> GetEnumerator() => ((IEnumerable<T>)items).GetEnumerator();
}

In [ ]:
NotEnumerableBox<int> box = new NotEnumerableBox<int>(1, 2, 3, 4);

foreach(var item in box)
{
    Console.WriteLine(item);
}

**!!! Это исключительный случай !!!**

## 3. IEnumerator

Интерфейсы `IEnumerator` и `IEnumerator<T>` определяются так:

```csharp
public interface IEnumerator<out T> : IEnumerator, IDisposable
{
    T Current { get; }
}

public interface IEnumerator
{
    object? Current { get; }

    bool MoveNext();

    void Reset();
}

public interface IDisposable
{
    void Dispose();
}

```

На самом деле конструкция
```csharp
foreach(var item in box)
{
    Console.WriteLine(item);
}
```
раскрывается приблизительно в такое 
```csharp
IEnumerator<int> enumerator = box.GetEnumerator();
while (enumerator.MoveNext()) // Пока не дошли до конца
{
    int objectName = enumerator.Value; // Берём следующее значение

    // Сюда подставляется тело foreach
    Console.WriteLine(item);
}
enumerator.Dispose(); // Для обобщённого варианта
```


## 4. yield

Стоит заметить, что `IEnumerable` *не обязывает коллекцию существовать в памяти*. Коллекция может генерироваться *на лету*.

Ключевое слово `yield` ("дать", "уступать") позволяет удобно описывать **генераторы коллекций**.

`yield` можно использовать только в методах, возвращающих `IEnumerable`, `IEnumerable<T>`, `IEnumerator` или `IEnumerator<T>`.

In [ ]:
IEnumerable<int> GetFirstFibonacci(int n)
{
    int curr = 1;
    int next = 1;
    for(int i = 0; i < n; i++)
    {
        yield return curr;
        next += curr;
        curr = next - curr;
    }
}

In [ ]:
foreach(var val in GetFirstFibonacci(10))
{
    Console.Write(val + " ");
}

Можно представлять, что `yield return` приостанавливает выполнение метода, сохраняя его состояние.

Также существует конструкция `yield break`, позволяющая прервать генерацию коллекции. Полезна как некоторый аналог `while`.

In [ ]:
IEnumerable<int> TakeWhileNotGreaterThan(IEnumerable<int> collection, int limit)
{
    foreach(var item in collection)
    {
        if(item > limit)
        {
            // Если значение превысило заданное, 
            // прекращаем генерировать коллекцию
            yield break; 
        }
        yield return item;
    }
}

In [ ]:
TakeWhileNotGreaterThan(new int[]{1, 4, 2, 5, 2, 1, 3}, 4)

**Обработка исключений:**
- Оператор `yield return` нельзя размещать в блоке `try-catch`. 

- Оператор `yield return` можно размещать в блоке `try` оператора `try-finally`.

- Оператор `yield break` можно размещать в блоке `try` или `catch`, но не в блоке finally.

А ещё нельзя использовать `yield` в лямбда-выражениях и анонимных методах.

# LINQ

Коллекции могут быть по-разному устроены, но операции над ними зачастую очень схожие:
- преобразовать каждый элемент
- отфильтровать элементы
- найти минимум/максимум/среднее/сумму...
- отсортировать по признаку
- сгруппировать по признаку (получить коллекцию из коллекций, каждая из которой соответствует уникальному значению признака)
- соединить по признаку
- ...

**LINQ (Language INtegrated Query)** представляет собой набор **методов расширений** над типом `IEnumerable<T>`, предоставляющий кучу полезных операций над <ins>любыми</ins> типами, реализующими `IEnumerable<T>`.

Ещё есть специальный синтаксис запросов, которым никто не пользуется. Почему: 1) меньше функций; 2) всё равно преобразуются в методы расширения.

!!! ВСЕ ОПЕРАЦИИ ВОЗВРАЩАЮТ НОВУЮ ПОСЛЕДОВАТЕЛЬНОСТЬ !!!

## Ключевые операции


- `Where(фильтр)` - фильтрует коллекцию по переданному предикату
- `Select(преобразование)` - преобразовывает каждый элемент последовательности

In [ ]:
void Print<T>(IEnumerable<T> collection)
{
    foreach(var item in collection)
    {
        Console.Write(item.ToString() + " ");
    }
    Console.WriteLine();
}

In [ ]:
var collection = Enumerable.Range(0, 10);

var result = collection.Where(x => x % 3 == 0).Select(x => x*x*x);

Print(collection);
Print(result);

## Сортировка (элементы должны быть `IComparable`)

- `OrderBy()` - сортировка по возрастанию
- `OrderByDescending()` - сортировка по убыванию
- `ThenBy()` - вторичная сортировка по возрастанию (сортируются элементы, значения которых совпали в пред. сортировке)
- `ThenByDescending()` - вторичная сортировка по убыванию (сортируются элементы, значения которых совпали в пред. сортировке)

In [ ]:
class Person
{
    public int Age { get; set; }
    
    public string Name { get; set; }
}

var people = new Person[] 
{
    new Person { Age = 34, Name = "Andrew" },
    new Person { Age = 23, Name = "Alice" },
    new Person { Age = 34, Name = "Bob" },
    new Person { Age = 25, Name = "Alice" }
};

In [ ]:
people.OrderBy(x => x.Age)

In [ ]:
people.OrderBy(x => x.Age).ThenByDescending(x => x.Name)

## Агрегирующие операции 
*схлопывающие последовательность в одно значение*

- `Count()` - число объектов (можно передать предикат)
- `Min()` - минимальное значение (можно передать предикат)
- `Max()` - максимальное значение (можно передать предикат)
- `Average()` - среднее **числовое** значение (можно передать предикат)
- `Sum()` - суммарное **числовое** значение (можно передать предикат)
- `Aggregate()` - обобщённая агрегирующая функция

In [ ]:
var collection = Enumerable.Range(1, 5);
Print(collection)

In [ ]:
collection.Count()

In [ ]:
collection.Count(x => x % 2 == 0)

In [ ]:
collection.Min()

In [ ]:
collection.Min(x => 2 * (x + 3))

In [ ]:
collection.Max()

In [ ]:
collection.Max(x => 2 * (x + 3))

In [ ]:
collection.Average()

In [ ]:
collection.Average(x => x % 3)

In [ ]:
collection.Sum()

In [ ]:
collection.Sum(x => x % 3)

In [ ]:
// Перемножение всех элементов
collection.Aggregate((prev, next) => prev * next)

## Пропуск/получение N элементов

- `Take(N)` - берёт первые N элементов последовательности
- `TakeWhile(condition)` - берёт элементы последовательности пока выполняется условие
- `Skip(N)` - пропускает первые N элементов последовательности
- `SkipWhile(condition)` - пропускает первые элементы последовательности пока выполняется условие

## Операции для работы с последовательностями как со множествами

- `Contains(obj)` - проверка на содержание объекта в последовательности
- `Except(otherCollection)` - разность множеств
- `Union(otherCollection)` - объединение множеств

**Подводный камень: `Union` и `Except` удаляют дубликаты**

In [ ]:
int[] set1 = { 1, 2, 2, 3, 4 };
int[] set2 = { 3, 4, 4, 5, 6 };

In [ ]:
set1.Contains(2)

In [ ]:
set1.Contains(5)

In [ ]:
Print(set1.Except(set2))

In [ ]:
Print(set1.Union(set2))

## Объединение коллекций

- `Concat(otherCollection)` - объединяет несколько коллекций в одну (последовательно)
- `Zip(otherCollection)` - склеивает элементы двух коллекций в соответствии с определенным условием
- `Join(otherCollection)` - соединяет две коллекции по определенному признаку

In [ ]:
int[] marks1 = {3, 4, 5};
int[] marks2 = {1, 2, 3};

Print(marks1.Concat(marks2))

In [ ]:
int[] ages = {12, 15, 18, 21};
int[] heights = {150, 165, 176, 178};

ages.Zip(heights, (age, height) => new {Age = age, Height = height}) // <--- анонимный тип, нет скобочек после new

In [ ]:
class Employee
{
    public string Name { get; set; }
    
    public string Department { get; set; }
}

In [ ]:
var people = new Person[] 
{
    new Person { Age = 34, Name = "Andrew" },
    new Person { Age = 23, Name = "Alice" },
    new Person { Age = 34, Name = "Bob" },
    new Person { Age = 25, Name = "Alice" }
};

var employees = new Employee[]
{
    new Employee { Name = "Bob", Department = "OAC" },
    new Employee { Name = "Mark", Department = "BBC" },
    new Employee { Name = "Andrew", Department = "DCA" }
};

people.Join(
    employees, 
    per => per.Name,
    emp => emp.Name,
    (per, emp) => new { Age = per.Age, Name = per.Name, Department = emp.Department })

## Получение элемента

- `First()` - возвращает первый элемент. Если коллекция пустая - исключение
- `FirstOrDefault()` - возвращает первый элемент. Если коллекция пустая - возвращает `null`


- `Last()` - возвращает последний элемент. Если коллекция пустая - исключение
- `LastOrDefault()` - возвращает последний элемент. Если коллекция пустая - возвращает `null`


- `Single()` - возвращает единственный элемент. Если коллекция содержит не 1 элемент - исключение 
- `SingleOrDefault()` - возвращает последний элемент. Если коллекция содержит не 1 элемент - возвращает `null`


- `ElementAt()` - возвращает единственный элемент. Если коллекция содержит не 1 элемент - исключение 
- `ElementAtOrDefault()` - возвращает последний элемент. Если коллекция содержит не 1 элемент - возвращает `null`

## Прочие полезные операции 

- `Reverse()` - разворачивает коллекцию
- `Distinct()` - удаляет дублирующиеся элементы из коллекции
- `Any(condition)` - проверяет, что хотя бы для одного элемента выполняется условие (возвращает bool)
- `All(condition)` - проверяет, что для всех элементов выполняется условие (возвращает bool)